### This notebook's purpose is to get the CALCE data in a workable format (in a dataframe, cleaned, battery type labeled etc.)

### Goals: (Add to this)
1. Get files from CALCE website into dataframe(s)
2. Add label for every cycle which battery this data came from 
3. Find/Remove empty cells  (delete the whole row or should we do interpolation?) (Might not even have empty cells)


All CS2 cells underwent the same charging profile which was a standard constant current/constant voltage protocol with a constant current rate of 0.5C until the voltage reached 4.2V and then 4.2V was sustained until the charging current dropped to below 0.05A. Unless specified, the discharge cut off voltage for these batteries was 2.7V. All the CS2 cells were randomly numbered and named accordingly. Name 'CS2_n' was given for the nth numbered CS2 cell. 

#### Imports:

In [1]:
import scipy.io
import scipy.signal
import os 
import pandas as pd
from pandas import ExcelWriter
#also had to pip install openpyxl
import zipfile
import requests
import xlrd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import statistics as stats
from math import isclose
from pathlib import Path
%matplotlib inline

#### Notes:
To import .mat files into python:  
import scipy.io   
mat = scipy.io.loadmat('file.mat')  

To unzip files: look at SEDS HW2 (not sure if we will use this)


#### CS2 data (in excel format*)  
Look at CS2 data first. From the website, CS2_'n' was given to the nth numbered CS2 cell (identifies unique cells from eachother). Each cell was cycled multiple times. Excel files are named using the testing dates.  **Were the same cells cycled on different dates or was the continuous cycling data split into multiple excel files?**   

*CS2_8 and CS2_21 are in .txt format because they were cycled using a different system.

Starting with CS2_33, in excel file format

### From Robert's Notebook + Sarah's Edits:

In [2]:
import glob 
import os 
import pandas as pd
import requests

def get_data(filepath): 
    """Imports all data in given path"""
    rootdir = filepath
    file_list = [f for f in glob.glob(os.path.join(rootdir,'*.xlsx'))] #iterate through dir to get excel files 
    
    d = {} #initiate dict for data storage
    count = 0
    for file in file_list:
        count += 1
        name = os.path.split(file)[1].split('.')[0]
        data = pd.read_excel(file,1)
        new_set = {name : data}
        d.update(new_set)
        print("adding file " + str(count) + ' ' + str(name))
    return d
### There are 23 files in the CS2 directory, so we should have 23 entries in the dictionary - add unit test for this, super EASY check 


In [3]:
d = get_data('data/CS2_33/')

adding file 1 CS2_33_10_04_10
adding file 2 CS2_33_10_05_10
adding file 3 CS2_33_10_15_10
adding file 4 CS2_33_10_26_10
adding file 5 CS2_33_11_01_10
adding file 6 CS2_33_11_10_10
adding file 7 CS2_33_11_19_10
adding file 8 CS2_33_11_24_10
adding file 9 CS2_33_12_08_10
adding file 10 CS2_33_12_16_10
adding file 11 CS2_33_12_23_10
adding file 12 CS2_33_1_10_11
adding file 13 CS2_33_1_18_11
adding file 14 CS2_33_1_24_11
adding file 15 CS2_33_1_28_11
adding file 16 CS2_33_2_2_11
adding file 17 CS2_33_8_17_10
adding file 18 CS2_33_8_18_10
adding file 19 CS2_33_8_19_10
adding file 20 CS2_33_8_30_10
adding file 21 CS2_33_9_17_10
adding file 22 CS2_33_9_27_10
adding file 23 CS2_33_9_7_10


In [4]:
data1 = d['CS2_33_10_04_10']

In [5]:
data1['Cycle_Index'].max()

23

In [6]:
#iterate over keys of data to add a battery label and a column of None dV values
def add_label_dv_cols(dictionary):
    '''This adds the battery label based off of the keys in the input dictionary, and adds an empty dV column and an empty dQ/dV column'''     
    for keys in dictionary:
        data1 = dictionary[keys]
        # add a battery label corresponding to that battery's key. 
        data1['Battery_Label'] = keys
        #create column labeled 'dV'
        data1['dV'] = None
        data1['dQ/dV'] = None 

In [7]:
add_label_dv_cols(dictionary = d)

In [8]:
#separate out dataframes into cycles
def sep_cycles(dataframe):
    """This function separates out the cycles in the battery dataframe by grouping by the 'Cycle_Index' column, and putting them in a dictionary. """
    gb = dataframe.groupby(by = ['Cycle_Index'])
    cycle_dict = dict(iter(gb))
    return cycle_dict

In [9]:
test = sep_cycles(data1)
#test is a dictionary of cycles

In [10]:

#writer = ExcelWriter('NLToutput.xlsx')
#test[1].to_excel(writer)
#writer.save()

In [11]:
def save_sep_cycles_xlsx(cycle_dict, battname, path_to_folder):
    '''This saves the separated out cycles into different excel files, beginning with the battery name. Battname and path to folder must be strings.'''
    for i in range(1,len(cycle_dict)+1):
        writer = ExcelWriter(path_to_folder + battname + 'Cycle' + str(i) + '.xlsx')
        cycle_dict[i].to_excel(writer)
        writer.save()

In [20]:
save_sep_cycles_xlsx(test, 'NLTtest', 'NLT_Test/')

### Calculate dV and dq/dv and Clean Data 

In [26]:
def calc_dv_dqdv(cycle_df):
    '''This function calculates the dv and the dq/dv for a dataframe.'''
    cycle_df = cycle_df.reset_index(drop = True)
    for i in range(1,len(cycle_df)): 
        cycle_df.loc[i, ('dV')] = cycle_df.loc[i, ('Voltage(V)')] - cycle_df.loc[i-1, ('Voltage(V)')]
    #calculate dq/dv based off of discharge capacity - might change this later so user can choose to use charge or discharge cap. 
    cycle_df['dQ/dV'] = cycle_df['Discharge_Capacity(Ah)']/cycle_df['dV']
    return cycle_df

In [27]:
data2 = calc_dv_dqdv(data1)

In [28]:
data2.head()

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
0,1,30.000116,2010-09-27 14:12:48,30.000117,1,1,0.000000,3.451826,0.000000,0.0,0.000000,0.0,-0.000032,0.0,0,0,0,CS2_33_10_04_10,None,NaN
1,2,60.015409,2010-09-27 14:13:18,60.015410,1,1,0.000000,3.451826,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,0,0,CS2_33_10_04_10,0,NaN
2,3,90.030676,2010-09-27 14:13:48,90.030677,1,1,0.000000,3.451988,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,0,0,CS2_33_10_04_10,0.000161886,0
3,4,120.014848,2010-09-27 14:14:18,120.014848,1,1,0.000000,3.451826,0.000000,0.0,0.000000,0.0,-0.000032,0.0,0,0,0,CS2_33_10_04_10,-0.000161886,-0
4,5,150.030659,2010-09-27 14:14:48,30.015332,2,1,0.550025,3.584821,0.004585,0.0,0.016314,0.0,0.001264,0.0,0,0,0,CS2_33_10_04_10,0.132995,0


In [16]:
def drop_0_dv(cycle_df_dv): 
    '''Drop rows where dv=0 (or about 0) in a dataframe that has already had dv calculated. Then recalculate dv and dq/dv'''
    #this will clean up the data points around V = 4.2V (since they are holding it at 4.2V for a while).
    cycle_df_dv = cycle_df_dv.dropna(subset=['dQ/dV'])
    cycle_df_dv = cycle_df_dv.reset_index(drop = True)
    for i in range(1, len(cycle_df_dv)):
        if isclose(cycle_df_dv.loc[i, ('dV')], 0, abs_tol = 10**-3):
            cycle_df_dv = cycle_df_dv.drop(index = i)
    #reset index
    cycle_df_dv = cycle_df_dv.reset_index(drop = True)
    #recalculating dv after dropping rows
    for i in range(1, len(cycle_df_dv)): 
        cycle_df_dv.loc[i, ('dV')] = cycle_df_dv.loc[i, ('Voltage(V)')] - cycle_df_dv.loc[i-1, ('Voltage(V)')]
    #recalculate dq/dv  
    cycle_df_dv['dQ/dV'] = cycle_df_dv['Discharge_Capacity(Ah)']/cycle_df_dv['dV']
    return cycle_df_dv  

In [18]:
data3 = drop_0_dv(cycle_df_dv=data2)

In [23]:
data3.tail()

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
9753,10755,356498.076170,2010-10-01 17:14:23,750.381752,7,23,-0.549993,3.983482,24.624331,23.690529,97.876133,88.765393,-0.000065,0.09335,0,0,0,CS2_33_10_04_10,-0.00372601,-6358.16
9754,10756,356528.091502,2010-10-01 17:14:53,780.397083,7,23,-0.550173,3.979756,24.624331,23.695116,97.876133,88.783657,-0.000065,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6359.8
9755,10757,356558.106839,2010-10-01 17:15:23,810.412421,7,23,-0.550173,3.976031,24.624331,23.699703,97.876133,88.801903,-0.000097,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6361.03
9756,10758,356588.107248,2010-10-01 17:15:53,840.412830,7,23,-0.550173,3.972305,24.624331,23.704288,97.876133,88.820123,-0.000097,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6362.26
9757,10759,356618.121556,2010-10-01 17:16:23,870.427138,7,23,-0.550173,3.968579,24.624331,23.708875,97.876133,88.838334,-0.000129,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6363.49


### Separate by charge/discharge 
#### We want to take each cycle as an individual, but we also want to separate into charge+discharge just to do gaussian fitting easier, but still take charge+discharge as one set. So after cycles are separated out to be treated as individuals, we will run above code to smooth, separate them into charge and discharge (could reverse those two steps) and then do gaussian fitting to find peak descriptors. 

In [24]:
data4 = pd.read_excel('NLT_Test/NLTtestCycle11.xlsx')
#as an example of one cycle

In [30]:
data5 = calc_dv_dqdv(data4)

In [ ]:
def sep_char_dis(df_dqdv):
    '''Takes a dataframe of one cycle with calculated dq/dv and separates into charge and discharge differential capacity curves'''
    
    
    

In [22]:
data.head()

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
4770,4771,156587.110342,2010-09-29 09:42:17,30.015269,1,11,0.000000,3.156191,10.689637,10.700286,42.453113,40.078347,0.000583,0.095613,0,0,0,CS2_33_10_04_10,NaN,NaN
4771,4772,156617.125821,2010-09-29 09:42:47,60.030749,1,11,0.000000,3.173686,10.689637,10.700286,42.453113,40.078347,0.000421,0.095613,0,0,0,CS2_33_10_04_10,NaN,NaN
4772,4773,156647.140885,2010-09-29 09:43:17,90.045813,1,11,0.000000,3.186970,10.689637,10.700286,42.453113,40.078347,0.000356,0.095613,0,0,0,CS2_33_10_04_10,NaN,NaN
4773,4774,156677.109273,2010-09-29 09:43:47,120.014200,1,11,0.000000,3.197175,10.689637,10.700286,42.453113,40.078347,0.000292,0.095613,0,0,0,CS2_33_10_04_10,NaN,NaN
4774,4775,156707.129010,2010-09-29 09:44:17,30.015288,2,11,0.549845,3.421047,10.694221,10.700286,42.468538,40.078347,0.002527,0.095613,0,0,0,CS2_33_10_04_10,NaN,NaN


### Plot code

New Column- To calculate dq/dv do discharge/charge capacity/ dv. calculate dv first. Ask robert whether to use discharge or charge capacity to calculate this. or do both and compare. Maybe ask user which capacity to use between discharge and charge - depends on system (i.e. Maccor) , which anode or cathode they are looking at etc. 

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 7:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['dQ/dV'])

### Filter Sav Golay

In [ ]:
data1['dV'].describe()

In [ ]:
unfilt = pd.concat([data1['dQ/dV-dis']])

In [ ]:
unfiltar = unfilt.values
#converts into an array 

In [ ]:
data1['filtered dq/dv'] = scipy.signal.savgol_filter(unfiltar, 21, 3)

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 6:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['filtered dq/dv'])